내가 앞서서 한 것들은 모두 잘못된 것이었다.. \
가장 정확하고 빠르게 확인하는 방법은 mipav에서 reorientation을 해보고, \
affine이 어떻게 바뀌었는지 본 뒤에 그걸 구현하는 것이다!

coronal에는 문제가 없는데 sagittal에서 mipav가 보여주는 affine과 python에서 출력하는 affine이 다르다. \
이유는 모르겠지만, 나중에 다른 데이터를 reorientation하게 되면 이 코드를 그대로 적용하지 말고, mipav와 비교해보자. 

In [ ]:
# load Nifti Image
import os
import nibabel as nib
import numpy as np
from glob import glob
import shutil
from copy import deepcopy

In [ ]:
# affine 출력하는 코드
filepath = "C:/Users/dain5/Downloads/9081096/registered/9081096_T1GD_registered_mipav.nii.gz"

img = nib.load(filepath)

img_data = img.get_fdata()
affine = np.empty_like(img.affine)
affine[:] = img.affine
print(img.affine)

In [ ]:
filepath = "C:/Users/dain5/Downloads/8861710/8861710_T1GD.nii.gz"
path = "C:/Users/dain5/Downloads/8861710/"
file = "T1GD_registered.nii"

img = nib.load(filepath)

img_data = img.get_fdata()
img_data = img_data.reshape(img_data.shape[0], img_data.shape[1], img_data.shape[2])
affine = np.empty_like(img.affine)
affine[:] = img.affine

orientation = nib.aff2axcodes(img.affine)
target_orientation = ('L', 'P', 'S')
print(img.affine)
if orientation == ('P', 'I', 'L'): # sagittal
    # filp
    img_flipped = np.flip(img_data, axis=1)
    affine[:,1]*=-1
    if affine[2,-1] > 0:
        affine[2,-1] = affine[2,-1] - img_data.shape[1] + 1

    elif affine[2,-1] < 0:
        affine[2,-1] = affine[2,-1] + img_data.shape[1] - 1
    
    # transpose
    img_T = img_flipped.transpose(2, 0, 1)
    affine[:,[0,1,2]] = affine[:,[2,0,1]]
#    affine[[0,1,2],-1] = affine[[2,0,1],-1]

elif orientation == ('L', 'I', 'P'): # coronal
    # flip
    img_flipped = np.flip(img_data, axis=1)
    affine[:,1]*=-1
    if affine[1,-1] > 0:
        affine[1,-1] = affine[1,-1] - img_data.shape[1] + 1

    elif affine[1,-1] < 0:
        affine[1,-1] = affine[1,-1] + img_data.shape[1] - 1


    # transpose
    img_T = img_flipped.transpose(0, 2, 1)
    affine[:,[1,2]] = affine[:,[2,1]]
    affine[[1,2],-1] = affine[[2,1],-1]


elif orientation == ('L', 'P', 'S'): # axial
    print("Image orientation is correct. The error was due to another issue.")
    
else:
    print("Image doesn't belong to coronal, sagittal, and axial. Check the orientation of Image")


print(affine)
print("Orientation {} is changed to {}".format(orientation, nib.aff2axcodes(affine)))
# reorient된 파일을 저장하려면 아래 코드를 모두 uncomment하세요.
ni_img = nib.Nifti1Image(img_T, affine)
outPath = os.path.join(path, file.split('.')[0] + '_AX.nii')
nib.save(ni_img, outPath)
print(ni_img.affine)